## 코드 실행 방법 : 컨트롤 + 앤터

In [ ]:
# 코드 실행해서 다운로드(다운은 한번만 하면 됨)
!pip install flask
!pip install xgboost

In [1]:
# 실행 후 밑에 url 클릭

# -*- coding : utf-8 -*- 

# 필요한 라이브러리 
from flask import Flask, render_template, request
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

# 데이터 불러오기
real = pd.read_csv("모기데이터2.csv")
train = pd.read_csv("train4.csv")
test = pd.read_csv("test4.csv")

# 데이터 수정
date = test['date']
del train['date']
del test['date']

# train, test 값 지정
X_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]
X_test = test
y_test = pd.read_csv('result4.csv')

# 모델 파라미터 설정 및 적용
model = XGBRegressor(base_score=0.5, colsample_bylevel=0.88, colsample_bynode=1,
                         gamma=2, gpu_id=-1, n_estimators=130,
                         interaction_constraints='', learning_rate=0.019, 
                         max_depth=3, subsample=1, validate_parameters=1) 
model.fit(X_train, y_train)

mid = 2500

# 플라스크 실행
app = Flask(__name__)

@app.route("/", methods = ['GET','POST'])
def index() :
    if request.method == 'GET' :
        return render_template('index.html')
    if request.method == 'POST' :
        month_input = float(request.form['month_input'])
        day_input = float(request.form['day_input'])
        temp_input = float(request.form['temp_input'])
        htemp_input = float(request.form['htemp_input'])
        ltemp_input = float(request.form['ltemp_input'])
        dcross_input = float(request.form['dcross_input'])
        rain_input = float(request.form['rain_input'])
        wind_input = float(request.form['wind_input'])
        humi_input = float(request.form['humi_input'])
        cloud_input = float(request.form['cloud_input'])
        sunlight_input = float(request.form['sunlight_input'])
        sunset_input = float(request.form['sunset_input'])
        
    price_value = 0
    
    data = (month_input, day_input, temp_input, htemp_input, ltemp_input,dcross_input, 
            rain_input, wind_input, humi_input, cloud_input, sunlight_input, sunset_input)
    X_test = np.array(data)
    X_test = pd.DataFrame([X_test], columns=['month','day', 'temp', 'htemp', 'ltemp', 'dcross', 'rain', 'wind', 'humi', 'cloud', 'sunlight','sunset'])
    y_predict = model.predict(X_test)
    price_value = y_predict[0]
    
    if price_value <= mid*0.5 :
        price = '쾌적'
    elif price_value <= mid :
        price = '관심'
    elif price_value <= mid*1.5:
        price = '주의'
    else :
        price = '불쾌'
        
    return render_template('index.html', price = price, price_value = price_value)

if __name__ == "__main__" :
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
